In [2]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans, AgglomerativeClustering

# --- load ---
df = pd.read_csv("sales_data_sample.csv", encoding="unicode_escape")

# --- numeric features only (drop obvious non-numeric columns) ---
drop_if_present = ['OrderDate','OrderDateTime','Order ID','Order ID','Order ID','Address','ADDRESSLINE1','ADDRESSLINE2','STATE','POSTALCODE','PHONE','CustomerID']
df = df.drop([c for c in drop_if_present if c in df.columns], axis=1, errors='ignore')
X = df.select_dtypes(include=[np.number]).copy()
X = X.dropna(axis=1, how='all')  # drop empty numeric cols
X = X.fillna(X.median())

# --- scale ---
scaler = StandardScaler()
Xs = scaler.fit_transform(X)

# --- elbow: inertia for k=1..10 ---
ks = list(range(1, 11))
inertias = []
for k in ks:
    inertias.append(KMeans(n_clusters=k, random_state=42, n_init=10).fit(Xs).inertia_)

# --- automatic elbow detection: distance from line (first-last) ---
# points (k, inertia)
pts = np.column_stack((ks, inertias))
p1, p2 = pts[0], pts[-1]
# line vector
v = p2 - p1
# distances
v3 = np.hstack((v, 0))
pts3 = np.hstack((pts - p1, np.zeros((pts.shape[0], 1))))
distances = np.abs(np.cross(v3, pts3)[:, 2]) / np.linalg.norm(v3)
optimal_k = int(ks[np.argmax(distances)])
if optimal_k < 2:
    optimal_k = 2

print("Inertia by k:", dict(zip(ks, inertias)))
print("Detected elbow (optimal k):", optimal_k)

# --- final KMeans ---
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10).fit(Xs)
df['kmeans_cluster'] = kmeans.labels_

# --- hierarchical clustering (Agglomerative) ---
agg = AgglomerativeClustering(n_clusters=optimal_k).fit(Xs)
df['hier_cluster'] = agg.labels_

# --- outputs ---
print("\nKMeans cluster counts:\n", df['kmeans_cluster'].value_counts().sort_index().to_dict())
print("\nHierarchical cluster counts:\n", df['hier_cluster'].value_counts().sort_index().to_dict())

# compact cluster summary (means of numeric features)
print("\nKMeans cluster centers (in original scale):")
centers = scaler.inverse_transform(kmeans.cluster_centers_)
centers_df = pd.DataFrame(centers, columns=X.columns).round(3)
print(centers_df)

# # save labelled data if needed
# df.to_csv("sales_clusters_labeled.csv", index=False)
# print("\nSaved labeled data to sales_clusters_labeled.csv")


Inertia by k: {1: 25407.000000000004, 2: 20090.887012173356, 3: 16909.3272126169, 4: 14818.012273279457, 5: 13541.406778644374, 6: 12547.114818612226, 7: 11743.686897235177, 8: 11067.285484133725, 9: 10529.8644594963, 10: 10085.260683115193}
Detected elbow (optimal k): 4

KMeans cluster counts:
 {0: 478, 1: 948, 2: 663, 3: 734}

Hierarchical cluster counts:
 {0: 1094, 1: 690, 2: 478, 3: 561}

KMeans cluster centers (in original scale):
   ORDERNUMBER  QUANTITYORDERED  PRICEEACH  ORDERLINENUMBER     SALES  QTR_ID  \
0    10392.297           36.885     83.445            5.973  3747.880   1.368   
1    10251.919           35.880     98.117            6.524  4667.526   3.629   
2    10181.014           34.884     85.941            6.423  3467.217   1.485   
3    10250.628           33.097     63.045            6.752  2065.889   3.530   

   MONTH_ID   YEAR_ID     MSRP  
0     3.002  2005.000  100.167  
1     9.831  2003.567  127.541  
2     3.364  2003.565  100.665  
3     9.578  2003.590 